In [7]:
!pip install pandas matplotlib ipywidgets

In [8]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from ipywidgets import interact

In [9]:
# Load data from CSV
data_file = 'run_data.csv'  # Ensure your .csv file is uploaded to Binder
df = pd.read_csv(data_file)

# Convert 'Date' to datetime for proper plotting
df['Date'] = pd.to_datetime(df['Date'], format='%d/%m/%y', errors='coerce')

# Check if there are any invalid date values after conversion
if df['Date'].isnull().any():
    print("Warning: Some dates could not be parsed correctly.")

# Preprocess 'Avg Pace (min/km)' column to match hh:mm:ss format
if 'Avg Pace (min/km)' in df.columns:
    df['Avg Pace (min/km)'] = (
        df['Avg Pace (min/km)']
        .str.replace("'", ":", regex=False)
        .str.replace('"', '', regex=False)
    )
    df['Avg Pace (min/km)'] = df['Avg Pace (min/km)'].apply(lambda x: f"00:{x}" if x.count(':') == 1 else x)
    df['Avg Pace (timedelta)'] = pd.to_timedelta(df['Avg Pace (min/km)'])

# Set a consistent date format for the x-axis
date_format = mdates.DateFormatter('%d-%m-%Y')

In [10]:
# Weekly summaries
df['Week'] = df['Date'].dt.to_period('W')
weekly_distance = df.groupby('Week')['Distance (km)'].sum()
weekly_pace = df.groupby('Week')['Avg Pace (timedelta)'].mean()

In [11]:
# Interactive plot function
def plot_interactive(start_date='2024-12-01', end_date='2024-12-31'):
    start_date = pd.to_datetime(start_date)
    end_date = pd.to_datetime(end_date)
    filtered_df = df[(df['Date'] >= start_date) & (df['Date'] <= end_date)]
    
    # Plot distance over time
    plt.figure(figsize=(10, 5))
    plt.plot(filtered_df['Date'], filtered_df['Distance (km)'], marker='o', label='Distance (km)')
    plt.title('Distance Progress Over Time')
    plt.xlabel('Date')
    plt.ylabel('Distance (km)')
    plt.gca().xaxis.set_major_formatter(date_format)  # Apply date format to x-axis
    plt.gcf().autofmt_xdate()  # Rotate date labels for readability
    plt.grid(True)
    plt.legend()
    
    # Plot average pace over time
    if 'Avg Pace (timedelta)' in filtered_df.columns:
        plt.figure(figsize=(10, 5))
        plt.plot(filtered_df['Date'], filtered_df['Avg Pace (timedelta)'].dt.total_seconds() / 60, marker='o', label='Avg Pace (min/km)')
        plt.title('Average Pace Progress Over Time')
        plt.xlabel('Date')
        plt.ylabel('Pace (min/km)')
        plt.gca().xaxis.set_major_formatter(date_format)
        plt.gcf().autofmt_xdate()
        plt.grid(True)
        plt.legend()

    # Plot average heart rate over time
    if 'Avg Heart Rate (bpm)' in filtered_df.columns:
        plt.figure(figsize=(10, 5))
        plt.plot(filtered_df['Date'], filtered_df['Avg Heart Rate (bpm)'], marker='o', label='Avg Heart Rate')
        plt.title('Heart Rate Progress Over Time')
        plt.xlabel('Date')
        plt.ylabel('Heart Rate (bpm)')
        plt.gca().xaxis.set_major_formatter(date_format)
        plt.gcf().autofmt_xdate()
        plt.grid(True)
        plt.legend()

    # Plot calories burned over time (assuming the column 'Calories Burned' exists in your dataset)
    if 'Calories Burned' in filtered_df.columns:
        plt.figure(figsize=(10, 5))
        plt.plot(filtered_df['Date'], filtered_df['Calories Burned'], marker='o', label='Calories Burned')
        plt.title('Calories Burned Over Time')
        plt.xlabel('Date')
        plt.ylabel('Calories')
        plt.gca().xaxis.set_major_formatter(date_format)
        plt.gcf().autofmt_xdate()
        plt.grid(True)
        plt.legend()

    # Plot stride frequency over time (assuming the column 'Avg Stride Frequency (spm)' exists in your dataset)
    if 'Avg Stride Frequency (spm)' in filtered_df.columns:
        plt.figure(figsize=(10, 5))
        plt.plot(filtered_df['Date'], filtered_df['Avg Stride Frequency (spm)'], marker='o', label='Stride Frequency')
        plt.title('Stride Frequency Over Time')
        plt.xlabel('Date')
        plt.ylabel('Stride Frequency (steps/min)')
        plt.gca().xaxis.set_major_formatter(date_format)
        plt.gcf().autofmt_xdate()
        plt.grid(True)
        plt.legend()

    # Plot weekly distance
    plt.figure(figsize=(10, 5))
    weekly_filtered_distance = weekly_distance[(weekly_distance.index >= start_date.to_period('W')) & (weekly_distance.index <= end_date.to_period('W'))]
    plt.plot(weekly_filtered_distance.index.astype(str), weekly_filtered_distance, marker='o', label='Weekly Distance (km)')
    plt.title('Weekly Distance Progress')
    plt.xlabel('Week')
    plt.ylabel('Distance (km)')
    plt.xticks(rotation=45)
    plt.grid(True)
    plt.legend()

    # Plot weekly average pace
    plt.figure(figsize=(10, 5))
    weekly_filtered_pace = weekly_pace[(weekly_pace.index >= start_date.to_period('W')) & (weekly_pace.index <= end_date.to_period('W'))]
    plt.plot(weekly_filtered_pace.index.astype(str), weekly_filtered_pace.dt.total_seconds() / 60, marker='o', label='Weekly Avg Pace (min/km)')
    plt.title('Weekly Average Pace Progress')
    plt.xlabel('Week')
    plt.ylabel('Pace (min/km)')
    plt.xticks(rotation=45)
    plt.grid(True)
    plt.legend()

# Display all figures
    plt.show()

In [13]:
# Create the interactive widget for date range selection
interact(plot_interactive, start_date='2024-12-01', end_date='2024-12-31');

interactive(children=(Text(value='2024-12-01', description='start_date'), Text(value='2024-12-31', description…